In [1]:
import numpy as np
import tensorflow as tf

%matplotlib inline
import matplotlib.pyplot as plt

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [2]:
reset_graph()

n_inputs = 3
n_neurons = 5

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

Y0 = tf.tanh(tf.matmul(X0, Wx) + b)             
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)

init = tf.global_variables_initializer()
# Mini-batch        instance 0, instance 1, instance 2, instance 3
X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]]) # t = 3
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]]) # t = 1

with tf.Session() as sess:
    init.run()
    
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [3]:
print(Y0_val)

[[-0.0664006   0.9625767   0.68105793  0.7091854  -0.898216  ]
 [ 0.9977755  -0.719789   -0.9965761   0.9673924  -0.9998972 ]
 [ 0.99999774 -0.99898803 -0.9999989   0.9967762  -0.9999999 ]
 [ 1.         -1.         -1.         -0.99818915  0.9995087 ]]


In [4]:
print(Y1_val)

[[ 1.         -1.         -1.          0.4020025  -0.9999998 ]
 [-0.12210419  0.62805265  0.9671843  -0.9937122  -0.2583937 ]
 [ 0.9999983  -0.9999994  -0.9999975  -0.85943305 -0.9999881 ]
 [ 0.99928284 -0.99999815 -0.9999058   0.9857963  -0.92205757]]


## static_rnn()

In [5]:
n_inputs = 3
n_neurons = 5

reset_graph()

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, [X0, X1], dtype=tf.float32)

Y0, Y1 = output_seqs

In [13]:
init = tf.global_variables_initializer()
X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]])
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]])

from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "e:/tmp/chap14/tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

with tf.Session() as sess:
    file_writer = tf.summary.FileWriter(logdir, sess.graph) 
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})
    run_metadata = tf.RunMetadata()
    file_writer.add_run_metadata(run_metadata, "step")
    #file_writer.add_summary()
    file_writer.close()

In [7]:
Y0_val

array([[ 0.30741334, -0.32884315, -0.6542847 , -0.9385059 ,  0.52089024],
       [ 0.99122757, -0.9542541 , -0.7518079 , -0.9995208 ,  0.9820235 ],
       [ 0.9999268 , -0.99783254, -0.8247353 , -0.9999963 ,  0.99947774],
       [ 0.996771  , -0.68750614,  0.8419969 ,  0.9303911 ,  0.8120684 ]],
      dtype=float32)

In [13]:
Y1_val

array([[ 0.99998885, -0.99976057, -0.0667929 , -0.9999803 ,  0.99982214],
       [-0.6524943 , -0.51520866, -0.37968948, -0.5922594 , -0.08968379],
       [ 0.99862397, -0.99715203, -0.03308626, -0.9991566 ,  0.9932902 ],
       [ 0.99681675, -0.9598194 ,  0.39660627, -0.8307606 ,  0.79671973]],
      dtype=float32)

### Packing sequence

In [14]:
n_steps = 2
n_inputs = 3
n_neurons = 5

In [16]:
reset_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
X_seqs = tf.unstack(tf.transpose(X, perm=[1,0,2]))
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, X_seqs, dtype=tf.float32)
outputs = tf.transpose(tf.stack(output_seqs), perm=[1, 0, 2])

In [17]:
init = tf.global_variables_initializer()

In [20]:
X_batch = np.array([
    # t = 0 t = 1
    [[0, 1, 2], [9, 8, 7]], # instance 0
    [[3, 4, 5], [0, 0, 0]], # instance 1
    [[6, 7, 8], [6, 5, 4]], # instance 2
    [[9, 0, 1], [3, 2, 1]], # instance 3
])

with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})

In [21]:
outputs_val

array([[[-0.45652324, -0.68064123,  0.40938237,  0.63104504,
         -0.45732826],
        [-0.9428799 , -0.9998869 ,  0.94055814,  0.9999985 ,
         -0.9999997 ]],

       [[-0.8001535 , -0.9921827 ,  0.7817797 ,  0.9971032 ,
         -0.9964609 ],
        [-0.637116  ,  0.11300927,  0.5798437 ,  0.4310559 ,
         -0.6371699 ]],

       [[-0.93605185, -0.9998379 ,  0.9308867 ,  0.9999815 ,
         -0.99998295],
        [-0.9165386 , -0.9945604 ,  0.896054  ,  0.99987197,
         -0.9999751 ]],

       [[ 0.9927369 , -0.9981933 , -0.55543643,  0.9989031 ,
         -0.9953323 ],
        [-0.02746338, -0.73191994,  0.7827872 ,  0.9525682 ,
         -0.9781773 ]]], dtype=float32)

In [22]:
np.transpose(outputs_val, axes=[1, 0, 2])[1]

array([[-0.9428799 , -0.9998869 ,  0.94055814,  0.9999985 , -0.9999997 ],
       [-0.637116  ,  0.11300927,  0.5798437 ,  0.4310559 , -0.6371699 ],
       [-0.9165386 , -0.9945604 ,  0.896054  ,  0.99987197, -0.9999751 ],
       [-0.02746338, -0.73191994,  0.7827872 ,  0.9525682 , -0.9781773 ]],
      dtype=float32)

## dynamic_rnn()

In [23]:
n_steps = 2
n_inputs = 3
n_neurons = 5

In [24]:
reset_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units = n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

In [25]:
init = tf.global_variables_initializer()

X_batch =  np.array([
        [[0, 1, 2], [9, 8, 7]], # instance 1
        [[3, 4, 5], [0, 0, 0]], # instance 2
        [[6, 7, 8], [6, 5, 4]], # instance 3
        [[9, 0, 1], [3, 2, 1]], # instance 4
    ])

with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})

In [27]:
outputs_val

array([[[-0.85115266,  0.87358344,  0.5802911 ,  0.8954789 ,
         -0.0557505 ],
        [-0.999996  ,  0.99999577,  0.9981815 ,  1.        ,
          0.37679607]],

       [[-0.9983293 ,  0.9992038 ,  0.98071456,  0.999985  ,
          0.25192663],
        [-0.7081804 , -0.0772338 , -0.85227895,  0.5845349 ,
         -0.78780943]],

       [[-0.9999827 ,  0.99999535,  0.9992863 ,  1.        ,
          0.5159072 ],
        [-0.9993956 ,  0.9984095 ,  0.83422637,  0.99999976,
         -0.47325212]],

       [[ 0.87888587,  0.07356028,  0.97216916,  0.9998546 ,
         -0.7351168 ],
        [-0.9134514 ,  0.3600957 ,  0.7624866 ,  0.99817705,
          0.80142   ]]], dtype=float32)

## Variable Sequence Length

In [28]:
n_steps = 2
n_inputs = 3
n_neurons = 5

reset_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)

In [30]:
seq_length = tf.placeholder(tf.int32, [None])
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32, sequence_length=seq_length)

In [31]:
init = tf.global_variables_initializer()

X_batch = np.array([
        # step 0     step 1
        [[0, 1, 2], [9, 8, 7]], # instance 1
        [[3, 4, 5], [0, 0, 0]], # instance 2 (padded with zero vectors)
        [[6, 7, 8], [6, 5, 4]], # instance 3
        [[9, 0, 1], [3, 2, 1]], # instance 4
    ])

seq_length_batch = np.array([2, 1, 2, 2])

with tf.Session() as sess:
    init.run()
    outputs_val, states_val = sess.run([outputs, states], feed_dict={X: X_batch, seq_length: seq_length_batch})

In [32]:
outputs_val

array([[[ 0.20123261,  0.8485545 ,  0.9367546 , -0.64976275,
          0.24727657],
        [ 0.9999977 ,  0.9999998 ,  1.        , -0.9948769 ,
          0.9394541 ]],

       [[ 0.98114127,  0.9996771 ,  0.9999982 , -0.9795104 ,
          0.7208298 ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ]],

       [[ 0.99972755,  0.9999994 ,  1.        , -0.9989911 ,
          0.916424  ],
        [ 0.99992263,  0.99983186,  1.        , -0.91878283,
          0.6966661 ]],

       [[ 0.9996366 ,  0.99158496,  0.99998814, -0.99899435,
          0.9882074 ],
        [ 0.9949612 ,  0.913094  ,  0.9943361 , -0.06307561,
          0.16509786]]], dtype=float32)

In [33]:
states_val

array([[ 0.9999977 ,  0.9999998 ,  1.        , -0.9948769 ,  0.9394541 ],
       [ 0.98114127,  0.9996771 ,  0.9999982 , -0.9795104 ,  0.7208298 ],
       [ 0.99992263,  0.99983186,  1.        , -0.91878283,  0.6966661 ],
       [ 0.9949612 ,  0.913094  ,  0.9943361 , -0.06307561,  0.16509786]],
      dtype=float32)

## Training sequence classifier

In [34]:
reset_graph()

n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10
learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)

loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

In [37]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("e:/tmp/data")
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

Extracting e:/tmp/data\train-images-idx3-ubyte.gz
Extracting e:/tmp/data\train-labels-idx1-ubyte.gz
Extracting e:/tmp/data\t10k-images-idx3-ubyte.gz
Extracting e:/tmp/data\t10k-labels-idx1-ubyte.gz


In [38]:
n_epochs = 100
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(iteration)
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X:X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X:X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

0 Train accuracy: 0.9260274 Test accuracy: 0.9202
1 Train accuracy: 0.92876714 Test accuracy: 0.9277
2 Train accuracy: 0.9506849 Test accuracy: 0.9504
3 Train accuracy: 0.95616436 Test accuracy: 0.957
4 Train accuracy: 0.9726027 Test accuracy: 0.9552
5 Train accuracy: 0.95616436 Test accuracy: 0.9657
6 Train accuracy: 0.9808219 Test accuracy: 0.9697
7 Train accuracy: 0.9917808 Test accuracy: 0.9703
8 Train accuracy: 0.97534245 Test accuracy: 0.9718
9 Train accuracy: 0.9780822 Test accuracy: 0.9722
10 Train accuracy: 0.9780822 Test accuracy: 0.9725
11 Train accuracy: 0.97534245 Test accuracy: 0.9725
12 Train accuracy: 0.9808219 Test accuracy: 0.9732
13 Train accuracy: 0.9068493 Test accuracy: 0.9219
14 Train accuracy: 0.95342463 Test accuracy: 0.9548
15 Train accuracy: 0.9616438 Test accuracy: 0.9657
16 Train accuracy: 0.9726027 Test accuracy: 0.9684
17 Train accuracy: 0.9808219 Test accuracy: 0.9724
18 Train accuracy: 0.97534245 Test accuracy: 0.9735
19 Train accuracy: 0.969863 Test ac

## Time Series